In [ ]:
class Trace_back_epochs(Callback):
    def __init__(self):
        super().__init__()
        self.epochs = []
        self.loss = []
        self.val_loss = []
        self.accuracy = []
        self.val_accuracy = []

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epochs.append(epoch + 1)
        self.loss.append(logs.get("loss"))
        self.val_loss.append(logs.get("val_loss"))
        self.accuracy.append(logs.get("accuracy"))
        self.val_accuracy.append(logs.get("val_accuracy"))


    def export_to_file(self, filepath):
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(filepath, "w") as fp:
            fp.write("Validation Loss:\n")
            fp.writelines('%s\n' % value for value in self.val_loss)
            fp.write("\nValidation Accuracy:\n")
            fp.writelines('%s\n' % value for value in self.val_accuracy)

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tabulate

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , recall_score , f1_score , classification_report , auc , roc_curve , ConfusionMatrixDisplay ,  roc_auc_score
from sklearn.utils import resample
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression , LogisticRegression
from sklearn.preprocessing import LabelEncoder , RobustScaler , StandardScaler
from sklearn.neural_network import MLPClassifier

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD ,Adam

plt.style.use("fivethirtyeight")
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/creditcardfraud/creditcard.csv
/kaggle/input/classification-test-file/Classification_Test_file.csv


# **Obtain the Data Set**

In [ ]:
creditCard = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [ ]:
creditCard.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
creditCard.sample(25)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
109395,71364.0,1.256634,0.388227,0.315844,0.688999,-0.330299,-1.062301,0.118363,-0.230313,-0.038034,...,-0.282242,-0.791741,0.123791,0.358121,0.231831,0.092843,-0.020027,0.031824,1.78,0
74056,55405.0,1.292317,0.140220,-0.197207,0.323588,0.089596,-0.130499,-0.160260,0.117665,0.123859,...,-0.332552,-1.078333,-0.007335,-0.939655,0.280504,0.155857,-0.040197,0.004994,2.69,0
122341,76524.0,-0.157467,2.158979,-4.258577,0.510864,2.809428,2.261109,-0.370936,1.550093,-0.549905,...,-0.129517,-0.519052,0.184903,0.648820,-0.211999,-0.388844,-0.021067,-0.059967,0.79,0
241895,151238.0,-0.650042,-0.058826,1.259346,0.297769,-0.804720,1.090243,-1.188712,-2.388140,0.770459,...,-1.464883,-0.453283,-0.353733,0.625838,1.325589,-0.549753,0.041847,0.223882,200.00,0
145412,86923.0,-0.246616,1.181016,-0.710945,-0.590141,0.552103,-0.575163,0.438263,0.531144,-0.353602,...,-0.245213,-0.757593,0.166595,0.576772,-0.488017,0.097795,0.091399,0.010884,1.98,0
111924,72430.0,-0.987278,1.497701,0.828519,0.001885,-0.388194,-0.805212,0.857727,0.024452,-0.262863,...,-0.299642,-0.654924,0.017159,0.294652,-0.088796,0.040828,-0.029621,-0.145301,69.99,0
258519,158690.0,2.085848,0.177487,-1.761855,0.377901,0.520699,-0.783871,0.204877,-0.245342,0.454810,...,-0.370981,-0.946580,0.316339,0.387947,-0.226184,0.180220,-0.060790,-0.031292,1.98,0
257944,158421.0,-0.850078,0.340200,0.018070,-0.903625,1.262558,1.055225,1.040310,0.440181,-0.128618,...,0.004852,-0.022146,0.083679,-1.562939,-0.109590,0.011221,0.104328,0.140642,128.70,0
223201,143285.0,2.019439,0.066202,-1.789726,0.569476,0.050605,-1.491212,0.175435,-0.283645,0.826815,...,0.185817,0.610967,-0.008402,-0.129028,0.178851,-0.094397,-0.011067,-0.030411,17.43,0
216609,140552.0,-2.290026,-0.772080,0.810270,-0.940607,0.700672,1.254000,0.040544,1.170115,0.147594,...,-0.143764,-1.041655,0.289068,-1.727704,0.245520,-0.170436,0.134792,-0.118533,186.48,0


# **Exploring the Data Set**

In [ ]:
creditCard.describe().T

,count,mean,std,min,25%,50%,75%,max
Time,284807.0,9.481386e+04,47488.145955,0.000000,54201.500000,84692.000000,139320.500000,172792.000000
V1,284807.0,1.168375e-15,1.958696,-56.407510,-0.920373,0.018109,1.315642,2.454930
V2,284807.0,3.416908e-16,1.651309,-72.715728,-0.598550,0.065486,0.803724,22.057729
V3,284807.0,-1.379537e-15,1.516255,-48.325589,-0.890365,0.179846,1.027196,9.382558
V4,284807.0,2.074095e-15,1.415869,-5.683171,-0.848640,-0.019847,0.743341,16.875344
V5,284807.0,9.604066e-16,1.380247,-113.743307,-0.691597,-0.054336,0.611926,34.801666
V6,284807.0,1.487313e-15,1.332271,-26.160506,-0.768296,-0.274187,0.398565,73.301626
V7,284807.0,-5.556467e-16,1.237094,-43.557242,-0.554076,0.040103,0.570436,120.589494
V8,284807.0,1.213481e-16,1.194353,-73.216718,-0.208630,0.022358,0.327346,20.007208
V9,284807.0,-2.406331e-15,1.098632,-13.434066,-0.643098,-0.051429,0.597139,15.594995


In [ ]:
creditCard.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
print(f"The Data Set contain {creditCard.shape[0]} Rows & {creditCard.shape[1]} Columns")
print(f"The Data Set contain {creditCard.duplicated().sum()} record duplicated") # Flag Here
print(f"Number of Nulls in each features :\n{creditCard.isna().sum()}")

The Data Set contain 284807 Rows & 31 Columns
The Data Set contain 1081 record duplicated
Number of Nulls in each features :
Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64


# **Imbalanced Features Resolving**

In [ ]:
X, y = creditCard.drop('Class', axis=1), creditCard['Class']

smote = SMOTE(random_state=42)
X_resampled_smote, y_resampled_smote = smote.fit_resample(X, y)

X_train, X_val, y_train, y_val = train_test_split( X_resampled_smote, y_resampled_smote, test_size=0.2, random_state=42)

scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


# **Modeling Our Data**

In [ ]:
modelAdam = Sequential()
modelAdam.add(Dense(512, input_dim=X_train.shape[1], activation='relu'))
modelAdam.add(Dense(256, activation='relu'))
modelAdam.add(Dense(128, activation='relu'))
modelAdam.add(Dense(len(np.unique(y_train)), activation='softmax'))

modelAdam.compile(
    optimizer=Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

traceEpochs = Trace_back_epochs()

modelAdam.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    verbose=1,
    callbacks=[Trace_back_epochs]
)

traceEpochs.export_to_file("Tracing Measures/Trial_1.txt")

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


I0000 00:00:1733873295.479291      82 service.cc:145] XLA service 0x7c6e68007100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733873295.479348      82 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1733873295.479352      82 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


  113/14216 ━━━━━━━━━━━━━━━━━━━━ 19s 1ms/step - accuracy: 0.8910 - loss: 0.2589 

I0000 00:00:1733873297.003576      82 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


14216/14216 ━━━━━━━━━━━━━━━━━━━━ 30s 2ms/step - accuracy: 0.9892 - loss: 0.0334 - val_accuracy: 0.9981 - val_loss: 0.0092
Epoch 2/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9987 - loss: 0.0058 - val_accuracy: 0.9992 - val_loss: 0.0039
Epoch 3/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9992 - loss: 0.0041 - val_accuracy: 0.9992 - val_loss: 0.0038
Epoch 4/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9993 - loss: 0.0035 - val_accuracy: 0.9990 - val_loss: 0.0067
Epoch 5/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9995 - loss: 0.0025 - val_accuracy: 0.9994 - val_loss: 0.0031
Epoch 6/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9995 - loss: 0.0023 - val_accuracy: 0.9996 - val_loss: 0.0017
Epoch 7/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9996 - loss: 0.0018 - val_accuracy: 0.9997 - val_loss: 0.0020
Epoch 8/50
14216/14216 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - accuracy: 0.9997 - lo

In [ ]:
with open("Trial 1.txt", "w") as fp:
    fp.writelines('%s\n' % value for value in plotly_callback.val_loss)
    fp.write("\nValidation Accuracy:\n")
    fp.writelines('%s\n' % value for value in plotly_callback.val_accuracy)

In [ ]:
existing_files = [f for f in os.listdir("/Phase_2/Predictions") if f.endswith('.csv')]
testFile = pd.read_csv("/kaggle/input/classification-test-file/Classification_Test_file.csv")
testFile_scaled = scaler.transform(testFile)
y_pred_multiclassAdam = np.argmax(modelAdam.predict(testFile_scaled), axis=1)
dictt = {
    "ID": [i for i in range(len(testFile))],
    "Class": y_pred_multiclassAdam
}
df = pd.DataFrame(dictt)
output_file_path = os.path.join(predictions_dir, f"Task-2 Trial ({len(existing_files) + 1 }).csv")
df.to_csv(output_file_path, index=False)